In [1]:
from __future__ import division
from salishsea_tools import rivertools
from salishsea_tools import nc_tools
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import arrow
import numpy.ma as ma
import sys
sys.path.append('/ocean/klesouef/meopar/tools/I_ForcingFiles/Rivers')
%matplotlib inline

In [2]:
# Constant and data ranges etc
year = 2015
smonth = 6
emonth = 6
startdate = arrow.get(year,smonth,15)
enddate = arrow.get(year,emonth,29)
print (startdate, enddate)

2015-06-15T00:00:00+00:00 2015-06-29T00:00:00+00:00


In [3]:
# get Fraser Flow data
filename = '/data/dlatorne/SOG-projects/SOG-forcing/ECget/Fraser_flow'
fraserflow = np.loadtxt(filename)
print (fraserflow)

[[ 2013.       10.       17.     1709.085]
 [ 2013.       10.       18.     1676.078]
 [ 2013.       10.       19.     1653.68 ]
 ..., 
 [ 2015.       10.       10.     1853.233]
 [ 2015.       10.       11.     2013.845]
 [ 2015.       10.       12.     1871.939]]


In [4]:
#Fraser watershed
pd = rivertools.get_watershed_prop_dict_long_fraser('fraser')
totalfraser = (pd['Fraser1']['prop'] + pd['Fraser2']['prop'] + 
               pd['Fraser3']['prop'] + pd['Fraser4']['prop'])

fraser has 10 rivers


In [5]:
# Climatology, Fraser Watershed
fluxfile = nc.Dataset('/ocean/jieliu/research/meopar/nemo-forcing/rivers/Salish_allrivers_monthly.nc','r')
climFraserWaterShed = fluxfile.variables['fraser'][:]
# Fraser River at Hope Seasonal Climatology (found in matlab using Mark's mean daily data)
climFraseratHope = (931, 878, 866, 1814, 4097, 6970, 5538, 3539, 2372, 1937, 1595, 1119)
NonHope = climFraserWaterShed - climFraseratHope
otherratio = 0.016
fraserratio = 1-otherratio

nonFraser = (otherratio * climFraserWaterShed.sum()/NonHope.sum()) * NonHope
afterHope = NonHope - nonFraser
print (pd['Fraser1']['i'],pd['Fraser1']['j'])

500 395


In [6]:
def calculate_daily_flow(r,criverflow):
    '''interpolate the daily values from the monthly values'''
    print (r.day, r.month)
    if r.day < 16:
        prevmonth = r.month-1
        if prevmonth == 0:
            prevmonth = 12
        nextmonth = r.month
    else:
        prevmonth = r.month
        nextmonth = r.month + 1
        if nextmonth == 13:
            nextmonth = 1
    fp = r - arrow.get(year,prevmonth,15)
    fn = arrow.get(year,nextmonth,15) - r
    ft = fp+fn
    fp = fp.days/ft.days
    fn = fn.days/ft.days
    print (ft, fp, fn)
    driverflow = fn*criverflow[prevmonth-1] + fp*criverflow[nextmonth-1]
    return driverflow

In [7]:
def write_file(r,flow,lat,lon,riverdepth):
    ''' given the flow and the riverdepth and the date, write the nc file'''
    directory = '.'
    # set up filename to follow NEMO conventions
    filename = 'NewRFraserCElse_y'+str(year)+'m'+'{:0=2}'.format(r.month)+'d'+'{:0=2}'.format(r.day)+'.nc'
    # print directory+'/'+filename
    nemo = nc.Dataset(directory+'/'+filename, 'w')
    nemo.description = 'Real Fraser Values, Daily Climatology for Other Rivers' 
    
    # dimensions
    ymax, xmax = lat.shape
    nemo.createDimension('x', xmax)
    nemo.createDimension('y', ymax)
    nemo.createDimension('time_counter', None)
    
    # variables
    # latitude and longitude
    nav_lat = nemo.createVariable('nav_lat','float32',('y','x'),zlib=True)
    nav_lat = lat
    x = nemo.createVariable('nav_lon','float32',('y','x'),zlib=True)
    nav_lon = lon
    # time
    time_counter = nemo.createVariable('time_counter', 'float32', ('time_counter'),zlib=True)
    time_counter.units = 'non-dim'
    time_counter[0:1] = range(1,2)
    # runoff
    rorunoff = nemo.createVariable('rorunoff', 'float32', ('time_counter','y','x'), zlib=True)
    rorunoff._Fillvalue = 0.
    rorunoff._missing_value = 0.
    rorunoff._units = 'kg m-2 s-1'
    rorunoff[0,:] = flow
    # depth
    rodepth = nemo.createVariable('rodepth','float32',('y','x'),zlib=True)
    rodepth._Fillvalue = -1.
    rodepth.missing_value = -1.
    rodepth.units = 'm'
    rodepth = riverdepth
    nemo.close()
    return

In [8]:
def fraser_correction(pd, fraserflux, r, afterHope, NonFraser, fraserratio, otherratio,
                      runoff):
    ''' for the Fraser Basin only, replace basic values with the new climatology after Hope and the
     observed values for Hope.  Note, we are changing runoff only and not using/changing river
     depth '''
    for key in pd:
        if "Fraser" in key:
            flux = calculate_daily_flow(r,afterHope) + fraserflux
            subarea = fraserratio
        else:
            flux = calculate_daily_flow(r,NonFraser)
            subarea = otherratio
        
        river = pd[key]
        runoff = rivertools.fill_runoff_array(flux*river['prop']/subarea,river['i'],
                          river['di'],river['j'],river['dj'],river['depth'],
                          runoff,np.empty_like(runoff))[0]
    return runoff

* Open climatology files

In [9]:
##open climatolgy file with modified fresh water point source 
clim_rivers_edit = nc.Dataset('/ocean/jieliu/research/meopar/river-treatment/rivers_month_nole.nc','r' )
criverflow_edit = clim_rivers_edit.variables['rorunoff']
lat = clim_rivers_edit.variables['nav_lat']
lon = clim_rivers_edit.variables['nav_lon']
riverdepth = clim_rivers_edit.variables['rodepth']
criverflow_edit[0,500,395]

6.0778766

In [11]:
for r in arrow.Arrow.range('day', startdate, enddate):
    print (r)
    driverflow = calculate_daily_flow(r, criverflow_edit)
    storeflow = calculate_daily_flow(r, criverflow_edit)
    step1 = fraserflow[fraserflow[:,0] == r.year]
    step2 = step1[step1[:,1] == r.month]
    step3 = step2[step2[:,2] == r.day]
#    print r.year, r.month, r.day, step3[0,3]
    runoff = fraser_correction(pd, step3[0,3] , r, afterHope, nonFraser, fraserratio, otherratio,
                              driverflow)
    write_file(r,runoff,lat,lon,riverdepth)
ig = 500
jg = 395
print (criverflow_edit[7:10,500,395], driverflow[ig,jg])
print (storeflow[ig,jg], driverflow[ig,jg])
#ig = 351; jg = 345
#print storeflow[ig,jg], driverflow[ig,jg]
#ig = 749; jg=123
#print storeflow[ig,jg], driverflow[ig,jg]

# jan 0, feb 1, mar 2, apr 3, may 4, jun 5
# jul 6, aug 7, sep 8

2015-06-15T00:00:00+00:00
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
15 6
31 days, 0:00:00 1.0 0.0
2015-06-16T00:00:00+00:00
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.03333333333333333 0.9666666666666667
16 6
30 days, 0:00:00 0.033333333333333